In [65]:
import pandas as pd
from scipy import sparse
import pandas as pd
import numpy as np
from numpy import var
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt
from scipy.stats import norm
import numpy as np
from scipy import stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statistics
import math
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import random
import copy
from gensim.parsing.preprocessing import remove_stopwords


#init
random.seed(1)

#load data
ratings = pd.read_csv('dataset/ratings.csv')
moviesPartial = pd.read_csv('dataset/movies.csv')
moviesFull = pd.read_csv('dataset/movie_dataset.csv')

#drop duplicates
moviesFull.drop_duplicates(subset = 'title', inplace=True)

#drop columns that do nothing
ratings = ratings.drop(['timestamp'], axis = 1)
moviesPartial = moviesPartial.drop(['genres'],axis=1)

#all the users are to be tested
userRatings = []
for i in range(1,611):
    userRatings.append(ratings[ratings['userId'] ==i])

#remove the year and keep the name
moviesPartial['title'] = moviesPartial['title'].map(lambda string: string[:-7])

#drop duplicates
moviesPartial.drop_duplicates(subset = 'title', inplace=True)


#helper functions:
def get_title_from_movieId(tmp):
    return moviesPartial[moviesPartial["movieId"]== tmp]["title"].to_string(index=False)

def checkTitle(tmp):
    if(tmp not in list(moviesFull["title"])):
        #The word "the" is being apended to the ends of titles
        tmp = tmp[-3:] +" "+ tmp[:-5]
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)
    else:
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)



#used to filter out movies that are not in both data sets
newUserTitles = []
newUserRatings = []
averageUserRating = 0
cnt =0
for userRating in userRatings:
    temp1 = []
    temp2 = []
    for rating, id in zip(list(userRating["rating"]), list(userRating["movieId"])):
        title = get_title_from_movieId(id)
        checked = checkTitle(title)
        if(checked != 'Series([], )'):
            temp1.append(checked)
            temp2.append(rating)
            averageUserRating += rating
            cnt+=1

    newUserTitles.append(temp1)
    newUserRatings.append(temp2)

averageUserRating/= cnt
    


infos = []
for user in newUserTitles:
    i = 0
    tempInfo = pd.DataFrame(columns = list(moviesFull.columns))
    for name in user:
        if(len(moviesFull[moviesFull["title"] == name]) !=0):
            tempInfo.loc[i] = moviesFull[moviesFull["title"] == name].iloc[0]
        i+=1
    infos.append(tempInfo)
    

features = ['keywords','cast','genres','director']


for info in infos:
    for feature in features:
        info[feature] = info[feature].fillna('')

colNumbers = []
for feature in features:   
    colNumbers.append(infos[0].columns.get_loc(feature))

fullDataList =[]


for frame in infos:
    fullDataList.append(frame.values.tolist())

combinedFeatures = []

for item in fullDataList:
    tmp = []
    for row in item:
        cleaned = remove_stopwords(row[colNumbers[0]]+" "+row[colNumbers[1]]+" "+row[colNumbers[2]]+" "+row[colNumbers[3]])
        tmp.append(cleaned)
    combinedFeatures.append(tmp)


def transform1(arr):
    ret = []
    for item in arr:
        if(item<=.5):
            ret.append(1/item)
        if(item>.5):
            ret.append(-1*(1/(1-item)))
    return ret

def transform2(arr):
    ret = []
    for item in arr:
        ret.append(1/item)
    return ret


def findAverageRating(arr1, arr2, title, ignore1, ignore2, ave):
    ret =0
    cnt =0
    single = True
    for index, (item1, item2) in enumerate(zip(arr1, arr2)):
        if(len(item1)!=len(item2)):
            print("diff", len(item1), len(item2))
        for j in range(len(item1)):
            if(title == item1[j] and (ignore1!=index or ignore2!=j)):
                ret+= item2[j]
                cnt+=1
                single = False

    if(single):
        return ave
    else:
        return ret/cnt



X = []
y = []

for i in range(len(combinedFeatures)):
    count_matrix = CountVectorizer().fit_transform(combinedFeatures[i]).toarray().tolist()
    randIndex = random.randint(0, len(count_matrix)-1)
    randTestItem = count_matrix[randIndex]
    del count_matrix[randIndex]

    cosine_sim = cosine_similarity(X = count_matrix ,Y = [randTestItem])

    ratings = copy.deepcopy(newUserRatings[i])
    randomRating = ratings[randIndex]
    del ratings[randIndex]


    similairities = np.reshape(cosine_sim,  (len(cosine_sim)))
    averageRating =  sum(ratings)/(len(ratings))

    tmp = []

    param = findAverageRating(newUserTitles, newUserRatings, newUserTitles[i][randIndex],i, randIndex, averageUserRating)
    tmp.append(param)
    tmp.append(averageRating)
    tmp.append(sum(similairities)/(len(similairities)))
    tmp.append(statistics.variance(ratings))
    tmp.append(statistics.variance(similairities))

    # ratingsZ = stats.zscore(ratings)
    # ratingsZ = [0 if math.isnan(x) else x for x in ratingsZ]
    # ratingsZ = [-item if item<0 else item for item in ratingsZ]
    # p_values = norm.sf(ratingsZ)
    # transformed2 = transform2(p_values)
    # sum2 = sum(transformed2)
    # simsZ  = stats.zscore(similairities)
    # simsZ = [-item if item<0 else item for item in simsZ]

    totalRating = 0


    for sim, rating in zip(similairities, ratings):
        totalRating += sim*(rating-averageRating)
    

    tmp.append(totalRating)
    X.append(tmp)
    y.append(randomRating)




In [76]:
#https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
#predicting imbd ratings website:
#https://jazpeng.github.io/predict_movie_ratings/

#need to cross validate!!!
#need to use krutosis and skewness:
#https://www.google.com/search?q=what+is+kurtosis&rlz=1C1ONGR_enUS1027US1027&oq=what+is+kurtosis&aqs=chrome..69i57j0i512j0i20i263i512j0i512l7.3563j0j7&sourceid=chrome&ie=UTF-8
#https://www.statisticshowto.com/probability-and-statistics/descriptive-statistics/skewness/#:~:text=The%20formula%20given%20in%20most,%E2%80%93%20Median)%20%2F%20Standard%20Deviation.




from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(X)
print(y)

regr = MLPRegressor(hidden_layer_sizes=(200, 200, 200),  random_state =1).fit(X_train, y_train)

score  = regr.score(X_test, y_test)
file = open("testing.txt", "a")
file.write("hidden_layer_sizes="+str(regr.hidden_layer_sizes)+", max_iter = "+str(regr.max_iter)+" " + str(score)+"\n")
file.close()

print("after nearal net regression:", score)
print(regr.max_iter)
print(regr.n_iter_)
print(regr.hidden_layer_sizes)

predicted = regr.predict(X_test)
for item, actual in zip(predicted, y_test):
    print("predicted:", item)
    print("actual:", actual)

[[3.2560975609756095, 4.363636363636363, 0.04167949551131953, 0.6274007682458387, 0.0026695513558354467, 0.09124091886536033], [3.95, 4.043478260869565, 0.06770729506301222, 0.5434782608695652, 0.003507670935774326, 6.525825018766432e-05], [2.872093023255814, 1.5294117647058822, 0.07185603833415556, 2.858455882352941, 0.007309959668710622, 1.6381279873308203], [4.2155172413793105, 3.451923076923077, 0.037641597634876005, 1.8617438386855862, 0.0021643585376757294, -0.6873973238231363], [3.8333333333333335, 3.566666666666667, 0.05399456773509661, 1.0126436781609196, 0.0013718774744848968, -0.02987108441762744], [3.591743119266055, 3.6370967741935485, 0.0612697277223213, 0.8672305271439811, 0.002142508910317742, -0.1331636444524524], [3.2837837837837838, 3.1083333333333334, 0.0530051442807298, 1.8579131652661065, 0.0023991914721097213, -0.04827538330496632], [3.43859649122807, 3.71875, 0.0597158898248902, 0.9828629032258065, 0.0012327408086946971, 0.103484879008545], [3.6845238095238093, 

In [78]:
reg = LinearRegression().fit(X_train, y_train)
score  = reg.score(X_test, y_test)

print(score)
print(reg.coef_)
predicted = regr.predict(X_test)
for item, actual in zip(predicted, y_test):
    print("predicted:", item)
    print("actual:", actual)



0.2591104641024269
[ 0.62206383  0.750706    4.00261497 -0.09104025 -5.96372958  0.15492384]
predicted: 3.348672064127326
actual: 2.5
predicted: 4.42558075715311
actual: 3.0
predicted: 4.433280707447917
actual: 4.5
predicted: 3.4164255904233833
actual: 4.0
predicted: 4.109951599066102
actual: 4.0
predicted: 4.626119983029079
actual: 5.0
predicted: 3.8007008564775537
actual: 2.0
predicted: 3.359100404678178
actual: 0.5
predicted: 4.5287330678727065
actual: 4.5
predicted: 3.4546707889929484
actual: 3.0
predicted: 3.684555764747725
actual: 4.0
predicted: 4.176342703245917
actual: 4.0
predicted: 3.225166896789493
actual: 3.0
predicted: 4.124788630673591
actual: 3.0
predicted: 2.3150049279587477
actual: 3.0
predicted: 4.068251886671497
actual: 4.5
predicted: 3.424993920471116
actual: 3.5
predicted: 3.462985977584996
actual: 3.0
predicted: 3.5625031968197516
actual: 3.0
predicted: 4.852114516495348
actual: 5.0
predicted: 4.056174948401183
actual: 3.5
predicted: 3.8854285468237895
actual: 4.0